## Exercise 1

如果你用完全相同的训练数据训练了五种不同的模型，并且它们都达到了95%的精度，那么你有机会将这些模型结合起来以得到更好的结果吗？如果是，如何处理？如果不是，为什么？

**答案**：

如果你已经训练了五个不同的模型并且它们都达到了 95% 的精度，你可以尝试将它们组合成一个投票集成，这通常会给你带来更好的结果。如果模型非常不同（例如，SVM 分类器、决策树分类器、逻辑回归分类器等），它会更好地工作。如果他们在不同的训练实例上接受训练（这就是 **bagging** 和 **pasting** 集成的全部意义），那就更好了，但如果不是这样，只要模型非常不同，这仍然有效。

## Exercise 2

硬投票和软投票分类器的区别有什么区别？

**答案**：

硬投票分类器只是计算集成中每个分类器的选票，并选择获得最多选票的类别。软投票分类器计算每个类别的平均估计类别概率，并选择概率最高的类别。这会赋予高置信度投票更大的权重并且通常表现得更好，但它仅在每个分类器都能够估计类别概率时才有效（例如，对于 Scikit-Learn 中的 SVM 分类器，您必须设置 **probability=True** ）。

## Exercise 3

是否有可能通过将一个 **bagging** 集成分布在多个服务器上来加速对它的训练？pasting 集成，boosting 集成，随机森林，或者堆叠集成怎么样？

**答案**：

很有可能通过将 bagging 集成分布在多个服务器上来加速它的训练，因为集成中的每个预测器都独立于其他预测器。出于同样的原因，pasting 集成和随机森林也是如此。然而，boosting 集成中的每个预测器都是基于前一个预测器构建的，因此训练必然是顺序的，并且通过在多个服务器上分布训练不会获得任何好处。关于堆叠集成，给定层中的所有预测器都是相互独立的，因此它们可以在多个服务器上并行训练。但是，一层的预测器只有在上一层的预测器都训练完后才能进行训练。

## Exercise 4

袋外评估的好处是什么？

**答案**：

通过袋外评估，bagging 集成中的每个预测器都使用未训练的实例（它们被保留）进行评估。这使得可以在不需要额外验证集的情况下对整体进行相当公正的评估。 因此，您有更多实例可用于训练，并且您的整体性能会稍微好一些。

## Exercise 5

是什么使 extra-trees 集成比常规的随机森林更随机？这种额外的随机性会有什么帮助呢？extre-trees 分类器比常规的随机森林慢还是更快？

**答案**：

当您在随机森林中种植一棵树时，仅考虑特征的随机子集以在每个节点处进行拆分。Extra-Trees 也是如此，但它们更进一步：它们不像常规决策树那样搜索最佳阈值，而是为每个特征使用随机阈值。这种额外的随机性就像一种正则化形式：如果随机森林过度拟合训练数据，Extra-Trees 可能会表现更好。此外，由于 Extra-Trees 不搜索可能的最佳阈值，因此它们的训练速度比随机森林快得多。但是，在进行预测时，它们既不比随机森林快也不慢。

## Exercise 6

如果你的AdaBoost集成欠拟合训练数据，你应该调整哪些超参数，如何调整？

**答案**：

如果您的 AdaBoost 集成欠拟合训练数据，您可以尝试增加估计器的数量或减少基本估计器的正则化超参数。您也可以尝试稍微降低学习率。

## Exercise 7

如果你的梯度增强集成过拟合训练集，你应该增加还是减少学习率？

**答案**：

如果您的 Gradient Boosting 集成过度拟合训练集，您应该尝试提高学习率。 您还可以使用提前停止来找到正确数量的预测变量（您可能有太多）。

## Exercise 8

加载 MNIST 数据集（在第3章中介绍），并将其拆分为训练集、验证集和测试集（例如，使用 50,000 个实例进行训练，10,000 个实例进行验证，10,000 个实例进行测试）。然后训练各种分类器，例如随机森林分类器、extre-trees 分类器和 SVM 分类器。接下来，尝试使用软投票或硬投票将它们组合成一个优于验证集上每个单独分类器的整体。找到一个后，在测试集上尝试一下。与单个分类器相比，它的性能好多少？

**答案**：

MNIST 数据集提前加载。数据集已经分成训练集（前 60,000 个实例）和测试集（最后 10,000 个实例），并且训练集已经洗牌。所以我们需要做的就是将前 50,000 个实例用于新训练集，接下来的 10,000 个实例用于验证集，最后 10,000 个实例用于测试集：

In [71]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True, as_frame=False)

X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]

X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]

X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

然后训练各种分类器，例如随机森林分类器、Extra-Trees 分类器和 SVM。

In [72]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [73]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)

mlp_clf = MLPClassifier(random_state=42)

In [74]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]

for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(random_state=42)
Training the ExtraTreesClassifier(random_state=42)
Training the LinearSVC(max_iter=100, random_state=42, tol=20)
Training the MLPClassifier(random_state=42)


In [75]:
[estimator.score(X_valid, y_valid) for estimator in estimators]

[0.9736, 0.9743, 0.8662, 0.9648]

线性 SVM 远远不如其他分类器。但是，让我们暂时保留它，因为它可能会提高投票分类器的性能。

接下来，尝试使用软投票分类器或硬投票分类器将分类器组合成一个在验证集上优于它们的整体。

In [76]:
from sklearn.ensemble import VotingClassifier

In [77]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [78]:
voting_clf = VotingClassifier(named_estimators)

In [79]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [80]:
voting_clf.score(X_valid, y_valid)

0.9752

**VotingClassifier** 克隆了每个分类器，并使用类索引而不是原始类名称作为标签来训练克隆。因此，为了评估这些克隆，我们还需要提供类索引。要将类转换为类索引，我们可以使用 **LabelEncoder** ：

In [81]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y_valid_encoded = encoder.fit_transform(y_valid)

然而，在 MNIST 的情况下，将类名转换为整数更简单，因为数字与类 ID 匹配：

In [82]:
import numpy as np

y_valid_encoded = y_valid.astype(np.int64)

现在让我们评估分类器克隆：

In [83]:
[estimator.score(X_valid, y_valid_encoded)
 for estimator in voting_clf.estimators_]

[0.9736, 0.9743, 0.8662, 0.9648]

让我们移除 SVM 以查看性能是否有所提高。可以通过使用 **set_params()** 将估算器设置为“drop”来删除估算器，如下所示：

In [84]:
voting_clf.set_params(svm_clf="drop")

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', 'drop'),
                             ('mlp_clf', MLPClassifier(random_state=42))])

这更新了估计器列表：

In [85]:
voting_clf.estimators

[('random_forest_clf', RandomForestClassifier(random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(random_state=42)),
 ('svm_clf', 'drop'),
 ('mlp_clf', MLPClassifier(random_state=42))]

但是，它没有更新训练过的估计器列表：

In [86]:
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 LinearSVC(max_iter=100, random_state=42, tol=20),
 MLPClassifier(random_state=42)]

In [87]:
voting_clf.named_estimators_

{'random_forest_clf': RandomForestClassifier(random_state=42),
 'extra_trees_clf': ExtraTreesClassifier(random_state=42),
 'svm_clf': LinearSVC(max_iter=100, random_state=42, tol=20),
 'mlp_clf': MLPClassifier(random_state=42)}

所以我们可以再次拟合 **VotingClassifier** ，或者只是从训练过的估计器列表中删除 SVM，在 estimators_ 和 named_estimators_ 中都进行相同的动作：

In [88]:
svm_clf_trained = voting_clf.named_estimators_.pop("svm_clf")

voting_clf.estimators_.remove(svm_clf_trained)

In [89]:
voting_clf.score(X_valid, y_valid)

0.9756

好一些！ SVM 会降低性能。现在让我们尝试使用软投票分类器。我们实际上不需要重新训练分类器，我们可以将 `voting` 设置为“`soft`”：

In [90]:
voting_clf.voting = "soft"

In [91]:
voting_clf.score(X_valid, y_valid)

0.9703

不，在这种情况下硬投票获胜。

一旦找到比单个预测器表现更好的集成，请在测试集上尝试它。与单个分类器相比，它的性能好多少？

In [92]:
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9722

In [93]:
[estimator.score(X_test, y_test.astype(np.int64))
 for estimator in voting_clf.estimators_]

[0.968, 0.9703, 0.9644]

投票分类器将最佳模型的错误率从大约 3% 降低到 2.7%，这意味着错误减少了 10%。

## Exercise 9

运行上一个练习中的各个分类器以对验证集进行预测，并使用生成的预测创建一个新的训练集：每个训练实例都是一个向量，其中包含所有分类器对图像的预测集，目标是图像的类。在这个新的训练集上训练分类器。恭喜——你刚刚训练了一个混合器blender，它与分类器一起形成了一个堆叠集成！现在评估测试集上的集成。对于测试集中的每个图像，使用所有分类器进行预测，然后将预测提供给混合器以获得整体预测。它与您之前训练的投票分类器相比如何？现在再次尝试使用 StackingClassifier。你有更好的表现吗？如果是这样，为什么？

**答案**：

In [94]:
X_valid_predictions = np.empty((len(X_valid), len(estimators)), dtype=object)

for index, estimator in enumerate(estimators):
    X_valid_predictions[:, index] = estimator.predict(X_valid)

In [95]:
X_valid_predictions

array([['3', '3', '3', '3'],
       ['8', '8', '8', '8'],
       ['6', '6', '6', '6'],
       ...,
       ['5', '5', '5', '5'],
       ['6', '6', '6', '6'],
       ['8', '8', '8', '8']], dtype=object)

In [96]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, 
                                            oob_score=True,
                                            random_state=42)

rnd_forest_blender.fit(X_valid_predictions, y_valid)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [97]:
rnd_forest_blender.oob_score_

0.9728

In [98]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=object)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [99]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [100]:
accuracy_score(y_test, y_pred)

0.9699

由于 StackingClassifier 使用 K-Fold 交叉验证，我们不需要单独的验证集，所以让我们将训练集和验证集加入一个更大的训练集：

In [101]:
X_train_full, y_train_full = X_mnist[:60_000], y_mnist[:60_000]

现在让我们在完整的训练集上创建和训练堆叠分类器：

In [102]:
from sklearn.ensemble import StackingClassifier

stack_clf = StackingClassifier(named_estimators,
                               final_estimator=rnd_forest_blender)

stack_clf.fit(X_train_full, y_train_full)

NameError: name 'StackingClassifier' is not defined

In [ ]:
stack_clf.score(X_test, y_test)

StackingClassifier 明显优于我们之前尝试的自定义堆叠实现！这主要有两个原因：

- 由于我们可以回收验证集，因此 StackingClassifier 在更大的数据集上进行了训练。
- 如果 predict_proba() 可用，它使用 predict_proba()，如果 decision_function() 可用，则使用 decision_function()，否则使用 predict()。 这为混合器提供了更细微的输入来处理。